In [1]:
import pandas as pd 
import os
import gzip
import csv
import json
import gzip
import matplotlib.pyplot as plt
import numpy as np
from textblob import TextBlob
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
!pip install transformers
!pip install Unidecode

# Prepare data

In [2]:
from google.colab import drive
drive.mount ('/content/gdrive')

Mounted at /content/gdrive


In [3]:
os.environ ['KAGGLE_CONFIG_DIR']='/content/gdrive/MyDrive/kaggle_dataset'

In [4]:
%cd /content/gdrive/MyDrive/kaggle_dataset

/content/gdrive/MyDrive/kaggle_dataset


In [ ]:
! kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
replace 0401_UkraineCombinedTweetsDeduped.csv.gzip? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Sentiment analysis

In [8]:
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [9]:
def get_sentiment(tweet):
    sentiment = TextBlob(tweet).sentiment
    return sentiment.polarity, sentiment.subjectivity

In [10]:
#f=gzip.open("UkraineCombinedTweetsDeduped_MAR26.csv.gzip",'rb')
#df = pd.read_csv(f,dtype={'coordinates':'str'})

In [11]:
def get_sentiment_from_file(filename):
  f=gzip.open(filename, 'rb')
  df = pd.read_csv(f,dtype={'coordinates':'str'})
  sentiment_df = df[['tweetid', 'text', 'hashtags', 'language']]
  sentiment_df = sentiment_df[sentiment_df['language']=='en']
  sentiment_df['hashtags'] = sentiment_df.hashtags.map(lambda x: [i['text'] for i in eval(x)])  # Keeping only hashtags
  sentiment_df['text'] = sentiment_df['text'].apply(preprocess)
  sentiment_df['sentiment'] = sentiment_df['text'].apply(get_sentiment)
  sentiment_df['polarity'] = sentiment_df['sentiment'].apply(lambda x:x[0])
  sentiment_df.drop('sentiment', axis=1, inplace=True)
  mean_df = sentiment_df['polarity'].mean()
  return mean_df

In [28]:
def get_sentiment_from_file_median(filename):
  f=gzip.open(filename, 'rb')
  df = pd.read_csv(f,dtype={'coordinates':'str'})
  sentiment_df = df[['tweetid', 'text', 'hashtags', 'language']]
  sentiment_df = sentiment_df[sentiment_df['language']=='en']
  sentiment_df['hashtags'] = sentiment_df.hashtags.map(lambda x: [i['text'] for i in eval(x)])  # Keeping only hashtags
  sentiment_df['text'] = sentiment_df['text'].apply(preprocess)
  sentiment_df['sentiment'] = sentiment_df['text'].apply(get_sentiment)
  sentiment_df['polarity'] = sentiment_df['sentiment'].apply(lambda x:x[0])
  sentiment_df.drop('sentiment', axis=1, inplace=True)
  median_df = sentiment_df['polarity'].median()
  return median_df

TextBlob

In [30]:
full_df = []
data_list = []
#polarity_df = pd.DataFrame(data=None,names=['Filename','Mean'])
for root, dirs, files in os.walk("/content/gdrive/MyDrive/kaggle_dataset/"):
    for file in files:
        if file.startswith("05"):
        #if "MAR" in file:
            print(f"Current file:{file}")
            polarity_mean = get_sentiment_from_file(file)
            data_list.append([file, polarity_mean])
full_polarity_df = pd.DataFrame(data_list)
full_polarity_df.to_csv("polarity_05_median.csv", index=False)

Current file:0501_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0502_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0503_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0504_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0505_to_0507_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0508_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0509_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0510_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0511_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0512_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0513_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0514_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0515_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0516_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0517_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0518_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0519_UkraineCombinedTweetsDeduped.csv.gzip
Current file:0520_UkraineCombinedTweetsD

In [31]:
print(data_list)

[['0501_UkraineCombinedTweetsDeduped.csv.gzip', 0.05055330423986192], ['0502_UkraineCombinedTweetsDeduped.csv.gzip', 0.05178920340551566], ['0503_UkraineCombinedTweetsDeduped.csv.gzip', 0.0505750563855955], ['0504_UkraineCombinedTweetsDeduped.csv.gzip', 0.07318636009225755], ['0505_to_0507_UkraineCombinedTweetsDeduped.csv.gzip', 0.08877025433726062], ['0508_UkraineCombinedTweetsDeduped.csv.gzip', 0.08159490558236669], ['0509_UkraineCombinedTweetsDeduped.csv.gzip', 0.07401171050548916], ['0510_UkraineCombinedTweetsDeduped.csv.gzip', 0.0634907976890543], ['0511_UkraineCombinedTweetsDeduped.csv.gzip', 0.07122552729493886], ['0512_UkraineCombinedTweetsDeduped.csv.gzip', 0.06953353828431369], ['0513_UkraineCombinedTweetsDeduped.csv.gzip', 0.06256227523853854], ['0514_UkraineCombinedTweetsDeduped.csv.gzip', 0.0917548819490514], ['0515_UkraineCombinedTweetsDeduped.csv.gzip', 0.0997436256547683], ['0516_UkraineCombinedTweetsDeduped.csv.gzip', 0.0785295697917196], ['0517_UkraineCombinedTweetsDe

In [32]:
list_df = pd.DataFrame(data_list)

In [33]:
print(list_df)

                                                    0         1
0          0501_UkraineCombinedTweetsDeduped.csv.gzip  0.050553
1          0502_UkraineCombinedTweetsDeduped.csv.gzip  0.051789
2          0503_UkraineCombinedTweetsDeduped.csv.gzip  0.050575
3          0504_UkraineCombinedTweetsDeduped.csv.gzip  0.073186
4   0505_to_0507_UkraineCombinedTweetsDeduped.csv....  0.088770
5          0508_UkraineCombinedTweetsDeduped.csv.gzip  0.081595
6          0509_UkraineCombinedTweetsDeduped.csv.gzip  0.074012
7          0510_UkraineCombinedTweetsDeduped.csv.gzip  0.063491
8          0511_UkraineCombinedTweetsDeduped.csv.gzip  0.071226
9          0512_UkraineCombinedTweetsDeduped.csv.gzip  0.069534
10         0513_UkraineCombinedTweetsDeduped.csv.gzip  0.062562
11         0514_UkraineCombinedTweetsDeduped.csv.gzip  0.091755
12         0515_UkraineCombinedTweetsDeduped.csv.gzip  0.099744
13         0516_UkraineCombinedTweetsDeduped.csv.gzip  0.078530
14         0517_UkraineCombinedTweetsDed